### Create ocr training data from yuva yolo data

In [1]:
import yaml
import glob
import shutil
from pathlib import Path
import pandas as pd
import uuid
import cv2
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
with open('data/yuva_yolo/dataset.yaml', 'r') as f:
    dataset_yaml = yaml.safe_load(f)

label_map = {} #id to text
for i, name in enumerate(dataset_yaml['names']):
  label_map[i+1] = str(name.split(" ")[1])

label_map

{1: '1',
 2: '3',
 3: '4',
 4: '6',
 5: '0',
 6: '2',
 7: '8',
 8: '9',
 9: '7',
 10: '5'}

In [10]:
train_images_dir = "data/yuva_yolo/images"
train_labels_dir = "data/yuva_yolo/labels"
output_dir = "data/ocr_training_data"

if Path(output_dir).exists():
  shutil.rmtree(output_dir)
Path(output_dir).mkdir(exist_ok=True)

with open(output_dir + "/gt.txt", "w+") as gt_file:
    for img_path in glob.iglob(train_images_dir + "/*.jpg"):
        label_path = Path(train_labels_dir) / Path(img_path).name.replace(
            ".jpg", ".txt"
        )

        if not Path(label_path).exists():
          continue

        df = pd.read_csv(
            label_path, header=None, names=["class", "x_c", "y_c", "w", "h"], sep=" "
        )
        df = df.sort_values("x_c")
        df["x_min"] = 0
        df["y_min"] = 0
        df["x_max"] = 0
        df["y_max"] = 0

        label = ""
        for i, row in df.iterrows():
            df.loc[i, "x_min"] = row["x_c"] - row["w"] / 2
            df.loc[i, "x_max"] = row["x_c"] + row["w"] / 2
            df.loc[i, "y_min"] = row["y_c"] - row["h"] / 2
            df.loc[i, "y_max"] = row["y_c"] + row["h"] / 2
            label += label_map[row["class"]]

        x_min, y_min, x_max, y_max = (
            min(df["x_min"]),
            min(df["y_min"]),
            max(df["x_max"]),
            max(df["y_max"]),
        )
        img = cv2.imread(img_path)
        h, w, _ = img.shape
        x_min, y_min, x_max, y_max = (
            int(x_min * w) - 10,
            int(y_min * h) - 10,
            int(x_max * w) + 10,
            int(y_max * h) + 10,
        )
        crop_img = img[y_min:y_max, x_min:x_max]

        name = str(uuid.uuid4()) + ".png"
        crop_h, crop_w, _ = crop_img.shape
        crop_img = cv2.resize(crop_img, (200, int((200/crop_w)*crop_h)), interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(f"{output_dir}/{name}", crop_img)
        label = list(label)
        label.insert(len(label) - 1, '.')
        label = ''.join(label)
        gt_file.write(name + "\t" + label + "\n")
